In [ ]:
import numpy as np

def one_dim_pw(xx, s, op2_func):
    """
    Powell's Quadratic Interpolation Method (One-Dimensional Search)

    Parameters:
    xx : numpy array
        Current starting point (vector).
    s : numpy array
        Search direction (vector).
    op2_func : function
        Objective function to minimize.
    """

    dela = 0.005
    alp0 = 0.01

    # Initialize arrays for 3 points
    alpha = [0.0] * 3
    y = [0.0] * 3

    # --- Point 1 ---
    alpha[0] = alp0
    x1 = xx + alpha[0] * s
    y[0] = op2_func(x1)

    # --- Point 2 ---
    alpha[1] = alpha[0] + dela
    x2 = xx + alpha[1] * s
    y[1] = op2_func(x2)

    # --- Bracketing Logic ---
    # Determine the location of the 3rd point based on the slope
    if y[1] >= y[0]:
        # Function is increasing, go backward
        alpha[2] = alpha[0] - dela
    else:
        # Function is decreasing, go forward further
        alpha[2] = alpha[0] + 2 * dela

    eps = 100.0
    delta = 100.0

    # --- Optimization Loop ---
    while eps > 0.001 or delta > 0.001:

        # Evaluate Point 3
        x3 = xx + s * alpha[2]
        y[2] = op2_func(x3)

        fmin = min(y)

        a0, a1, a2 = alpha[0], alpha[1], alpha[2]
        f0, f1, f2 = y[0], y[1], y[2]

        # Denominator for quadratic interpolation formula
        denom = 2 * ((a1 - a2)*f0 + (a2 - a0)*f1 + (a0 - a1)*f2)

        if denom == 0:
            # Avoid division by zero if points are collinear
            break

        # Numerator
        num = (a1**2 - a2**2)*f0 + (a2**2 - a0**2)*f1 + (a0**2 - a1**2)*f2

        # Optimal alpha (vertex of the parabola)
        alpha_opt = num / denom

        # --- Update Stopping Criteria ---
        # eps: change in the variable (alpha)
        # delta: change in the function value

        # Find index of the best point so far
        best_idx = np.argmin(y)
        best_alpha = alpha[best_idx]
        best_y = y[best_idx]

        eps = abs(alpha_opt - best_alpha)

        # Evaluate function at the new optimal prediction
        x_opt = xx + s * alpha_opt
        y_opt = op2_func(x_opt)
        delta = abs(best_y - y_opt)

        # --- Prepare for Next Iteration ---
        # Replace the point with the highest function value (worst point)
        # with the new optimal point to refine the parabola.
        worst_idx = np.argmax(y)
        alpha[worst_idx] = alpha_opt
        y[worst_idx] = y_opt

    return best_alpha

# --- Usage Example ---
if __name__ == "__main__":
    # Define a test function: f(x) = x1^2 + x2^2 - 4x1 - 4x2
    # Minimum is at [2, 2]
    def my_obj(x):
        return x[0]**2 + x[1]**2 - 4*x[0] - 4*x[1]

    # Starting point and direction
    x_start = np.array([0.0, 0.0])
    search_dir = np.array([1.0, 1.0]) # Searching along the diagonal

    al_opt = one_dim_pw(x_start, search_dir, my_obj)

    print(f"Optimal Step Size (alpha): {al_opt:.4f}")
    print(f"New Point: {x_start + al_opt * search_dir}")